# Setup Enviorment

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2
from tensorflow.keras.applications.convnext import ConvNeXtBase, ConvNeXtSmall, ConvNeXtTiny

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

2023-07-24 19:10:09.942389: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 19:10:09.980537: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 19:10:09.981833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 19:10:10.777450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Path to Dataset
path = '../FULL_COLOMBIA_v2/'
ignore_black = False

model_name = 'variational_autoencoder' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder' # 'ResNet50V2' #'ConvNeXtTiny'
encoder_backbone = 'ResNet50V2' # 'vit' # 'ResNet50V2' # 'ConvNeXtTiny'
latent_dim = 1024

target_size = (224, 224, 3)
model_input = (224, 224, 3)
band = 9 #None

# Model path _full_dataset
if target_size[2] != 1:
    model_path = f'Weights/{model_name}_{encoder_backbone}_{target_size[0]}_{latent_dim}_{target_size[2]}Bands_full_dataset.h5' 
else:
    model_path = f'Weights/{model_name}_{encoder_backbone}_{target_size[0]}_{latent_dim}_3Bands_full_dataset.h5' 

if target_size[2] == 1:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}_Per_Band/{target_size[0]}_band{band+1}.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
else:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}__{target_size[0]}_{latent_dim}_{target_size[2]}Bands.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_{target_size[0]}_{latent_dim}.csv'
        
if ignore_black:
    embeddings_path = embeddings_path.replace(".csv", "_no_black_images.csv")    
    
if 'FULL' in path:
    embeddings_path = embeddings_path.replace(".csv", "_full_dataset.csv")  

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, ignore_black=ignore_black, show_dirs=True, head=0)

Images in directories: 
76400
50573
41020
85001
47288
73268
68307
95001
54874
76834
5088
68547
73443
50313
68081
44430
23855
41001
86320
73408
47001
86865
68406
13244
25290
76892
5837
73585
76109
68001
15480
81736
66001
5679
25488
76147
68655
18753
5490
8001
23555
23001
73675
50006
76111
20001
20011
5079
5266
15469
18001
95025
68679
76364
73319
15753
54405
8758
5360
73449
41298
5212
20710
73411
13430
23466
25307
68276
25245
76520
20013
54498
5045
66170
70001
86568
85010
41396
41551
13688
5656


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001, encoder_backbone=encoder_backbone)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, encoder_backbone=encoder_backbone)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 
elif model_name == 'ConvNeXtTiny':
    cnn = ConvNeXtTiny(input_shape=model_input, include_top=False, weights='imagenet')  

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2', 'ConvNeXtTiny']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


2023-07-24 19:10:16.413119: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-07-24 19:10:16.662152: W tensorflow/c/c_api.cc:300] Operation '{name:'conv5_block3_2_conv/kernel/Assign' id:3389 op device:{requested: '', assigned: ''} def:{{{node conv5_block3_2_conv/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv5_block3_2_conv/kernel, conv5_block3_2_conv/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-24 19:10:22.807205: W tensorflow/c/c_api.cc:300] Operation '{name:'log_variance/bias/Assign' id:6218 op device:{requested: '', assigned: ''} def:{{{node log_variance/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOA

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1024)              27761152  
                                                                 
Total params: 27,761,152
Trainable params: 0
Non-trainable params: 27,761,152
_________________________________________________________________


In [6]:
#from huggingface_hub import push_to_hub_keras

#from huggingface_hub import push_to_hub_keras
#push_to_hub_keras(model, 'MITCriticalData/Sentinel-2_ConvNeXtTiny_Autoencoder_RGB_full_Colombia_Dataset', create_pr=1)

## Generate embeddings

In [7]:
if target_size[2] == 1:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
else:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size)
embeddings

,Municipality Code,Date,Embedding
0,76400,2017-07-30,"[-0.9534884, 2.13462, 0.55131733, 0.89277846, ..."
1,76400,2015-12-20,"[0.33966315, 0.7755328, -0.31368774, -0.763054..."
2,76400,2017-11-19,"[1.1717373, -0.9062659, -0.10839006, 0.3421441..."
3,76400,2016-11-20,"[-0.33863792, 0.07670107, -2.240027, -0.190735..."
4,76400,2017-07-16,"[-0.9837804, -1.6694198, 0.9699997, 0.2738762,..."
...,...,...,...
13279,5656,2017-04-16,"[1.622838, -0.94712687, -0.78327656, -0.488896..."
13280,5656,2018-12-02,"[-0.2810181, 2.079969, -1.9714534, 0.6072134, ..."
13281,5656,2018-05-20,"[-0.5985662, -0.8644047, 0.43456012, 0.0675232..."
13282,5656,2016-09-18,"[-1.2459834, 0.095033705, -0.44481027, -0.4961..."


In [8]:
save_embeddings_as_csv(df=embeddings, path=embeddings_path)

In [9]:
pd.read_csv(embeddings_path)

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,76400,2017-07-30,-0.953488,2.134620,0.551317,0.892778,0.495981,0.385583,1.360742,0.413684,...,-0.343929,-0.970159,0.984231,-1.321890,-0.657946,1.527903,-0.941590,0.196390,-0.288964,0.008276
1,76400,2015-12-20,0.339663,0.775533,-0.313688,-0.763054,0.659456,-1.057165,0.707124,-0.309920,...,-0.245230,-0.785068,0.490844,-0.181403,0.911504,-0.487966,0.465531,0.566769,-0.544578,0.839298
2,76400,2017-11-19,1.171737,-0.906266,-0.108390,0.342144,1.629970,-1.222789,-1.548845,-0.583422,...,0.872039,0.431017,-0.047641,-0.320425,1.050420,0.309978,-0.041067,0.992501,-1.372416,-0.832866
3,76400,2016-11-20,-0.338638,0.076701,-2.240027,-0.190735,-0.486981,-0.368004,0.832553,-0.795786,...,0.004048,1.170392,0.252880,-1.677973,0.005628,-0.955936,-0.509528,-0.596950,1.293229,0.339890
4,76400,2017-07-16,-0.983780,-1.669420,0.970000,0.273876,-1.220695,-1.340708,1.931429,0.917812,...,0.267635,-0.335373,-0.595033,-0.799997,-1.372569,-1.485848,-0.786285,0.678145,2.319721,-1.129631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13279,5656,2017-04-16,1.622838,-0.947127,-0.783277,-0.488897,0.689022,0.812694,-0.113106,-0.782928,...,-0.246657,0.256836,0.931840,-0.484294,0.358134,-1.024092,-1.085175,-0.552554,-0.689482,1.249919
13280,5656,2018-12-02,-0.281018,2.079969,-1.971453,0.607213,0.640338,0.002874,1.158266,1.178751,...,0.348990,-1.543749,-0.127294,-0.404585,-1.258923,-1.933398,0.246151,0.170459,-0.158349,0.669746
13281,5656,2018-05-20,-0.598566,-0.864405,0.434560,0.067523,1.697596,-0.269326,0.123171,-0.306739,...,-3.234091,-1.367425,1.694562,0.377775,-0.051732,1.786254,-0.823142,1.503675,0.168561,-2.018319
13282,5656,2016-09-18,-1.245983,0.095034,-0.444810,-0.496117,-1.367250,-0.331551,0.783521,-0.629933,...,1.097344,1.225518,1.443542,0.671977,-2.099221,-0.228025,0.332816,0.994921,-0.068402,0.460993
